In [1]:
import requests
from BeautifulSoup import BeautifulSoup # XML
from pattern import web
import re
import pandas as pd

In [2]:
def scrap_page(cityCode, sggCityCode = '-1'):
    print "fetching " , cityCode, sggCityCode
        
    target_url = 'http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml?electionId=0020160413&requestURI=%2FWEB-INF%2Fjsp%2Felectioninfo%2F0020160413%2Fvc%2Fvccp09.jsp&topMenuId=VC&secondMenuId=VCCP09&menuId=VCCP09&statementId=VCCP09_%232&electionCode=2&cityCode='+ cityCode + '&sggCityCode='+ sggCityCode +'&proportionalRepresentationCode=-1&townCode=-1&sggTownCode=-1'
    r = requests.get(target_url)# your code here. Get requests for the given URL.
    
    return r.text

In [3]:
def str2number(number_str):
        if number_str is None:
            return None
        else:
            number_str = re.sub("[^0-9]", "", number_str) 
            # return int(number_str)
            if(number_str == "") : 
                number_str = '1';
            return int(float(number_str))

In [4]:
def parsing(cityCode):
    html = scrap_page(cityCode)
    soup = BeautifulSoup(html)
    

    table = soup.find('table')
    rows = table.findAll('tr')

    df = pd.DataFrame([], columns=['city', 'sggCity', 'party', 'sex', 'age', 'percent'])
    nameRows = []
    for i in range(1,len(rows)):
        if not rows[i].find('td').text == '':
            nameRows.append(i)

    for i in nameRows :
        rowOfName = rows[i].findAll('td')
        rowOfPercent = rows[i+1].findAll('td')

        for j in range(4,14):
            try:
                name = str(rowOfName[j]).split('<br />')[1].split('</strong></td>')[0]
                if not name == '':
                    party = str(rowOfName[j]).split('<br />')[0].split('<strong>')[1]
                    percent = float(rowOfPercent[j].text.split('(')[1].split(')')[0])
                    sggCity = rowOfName[0].text
                    newRow = pd.DataFrame([[cityCodes[cityCode] ,sggCity, party, '', 0, percent]], columns=['city', 'sggCity', 'party', 'sex', 'age', 'percent'], index=[name])
                    df = df.append(newRow)
            except:
                pass
    return df

In [5]:
cityCode={}
cityCodes['1100']='서울특별시'
cityCodes['2600']='부산광역시'
cityCodes['2700']='대구광역시'
cityCodes['2800']='인천광역시'
cityCodes['2900']='광주광역시'
cityCodes['3000']='대전광역시'
cityCodes['3100']='울산광역시'
cityCodes['5100']='세종특별자치시'
cityCodes['4100']='경기도'
cityCodes['4200']='강원도'
cityCodes['4300']='충청북도'
cityCodes['4400']='충청남도'
cityCodes['4500']='전라북도'
cityCodes['4600']='전라남도'
cityCodes['4700']='경상북도'
cityCodes['4800']='경상남도'
cityCodes['4900']='제주특별자치도'

NameError: name 'cityCodes' is not defined

In [ ]:
cityCodes['4900']

In [ ]:
df = pd.DataFrame([], columns=['city', 'sggCity', 'party', 'sex', 'age', 'percent'])
for cityCode in cityCodes.keys():
    rslt = parsing(cityCode)
    df= df.append(rslt)
print df

In [ ]:
df.ix['안호영']['percent']

In [ ]:
sample=df[df.city == u'김제시부안군']
sample.loc[:,'ranking'] = sample['percent'].rank(ascending=0)
sample[sample.ranking <= 3]

In [ ]:
sample[sample.ranking <= 3].percent

In [ ]:
for cityCode in cityCodes.keys():
    print 'fetching ' +cityCode
    url='http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml?electionId=0020160413&requestURI=%2FWEB-INF%2Fjsp%2Felectioninfo%2F0020160413%2Fcp%2Fcpri03.jsp&topMenuId=CP&secondMenuId=CPRI03&menuId=CPRI03&statementId=CPRI03_%232&electionCode=2&cityCode='+cityCode+'&sggCityCode=0&proportionalRepresentationCode=-1&townCode=-1&sggTownCode=-1&dateCode=0&x=46&y=8'
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html)
    table = soup.findAll('table',{'class':'table01'})[0]
    tagList = table.findAll('tr')
    rowList = tagList[1:len(tagList)]
    lst_tmp=[]
    for i in range(len(rowList)):
        lst_tmp.append(rowList[i])
    r_area = [row.findAll('td')[6] for row in lst_tmp]
    r_sex = [row.findAll('td')[5] for row in lst_tmp]
    r_name = [row.findAll('td')[4] for row in lst_tmp]
    r_loc= [row.findAll('td')[0] for row in lst_tmp]
    
    for a in range(len(r_area)) :
        tmp=str(r_area[a])
        tmp=tmp.split('(')
        tmp2=tmp[1]
        tmp2=tmp2.split(')')
        age = int(tmp2[0][0])*10+int(tmp2[0][1])

        tmp3=str(r_name[a])
        tmp3=tmp3.split('<br/>')
        name=tmp3[0].split('\t')
        name=name[11].split('<br />')[0]

        tmp9=str(r_sex[a])
        tmp9=tmp9.split('>')
        sex=tmp9[1].split('<')[0]
        
        tmp7=str(r_loc[a]).decode('utf-8')
        tmp7=tmp7.split('>')
        sggCity=tmp7[1].split('<')[0]
        
        try : 
            df['sex'][(df.index == name) & (df.sggCity == sggCity)] = sex
            df['age'][(df.index == name) & (df.sggCity == sggCity)] = age
            #df.set_value(name, 'sex', sex)
            #df.set_value(name, 'age', age)
        except :
            pass
df

In [ ]:
df.ix['허영']

In [24]:
dfPoint=pd.DataFrame.from_csv('all.csv', encoding ='euc-kr')
dfPoint

,sggCity,queryPoint
name,,
박종희,수원시갑,58
이찬열,수원시갑,100
김재귀,수원시갑,21
김상민,수원시을,100
박혜련,수원시을,2
이대의,수원시을,4
김용남,수원시병,100
김영진,수원시병,78
박수영,수원시정,39
